### Get data

In [17]:
import pandas as pd
def get_pima_data():
    filename = 'data/pima-indians-diabetes.csv'
    df=pd.read_csv(filename, header=None)
    Y_train = df[8].values
    del df[8]
    X_train = df.values
    return X_train, Y_train

### Naive Bayes

In [18]:
from collections import Counter, defaultdict
import numpy as np

class NaiveBaseClass:
    def calculate_relative_occurences(self, list1):
        no_examples = len(list1)
        ro_dict = dict(Counter(list1))
        for key in ro_dict.keys():
            ro_dict[key] = ro_dict[key] / float(no_examples)
        return ro_dict
    
    def get_max_value_key(self,dic):
        v = np.array(list(dic.values()))
        k = np.array(list(dic.keys()))
        
        maxima = np.where(v==np.max(v))[0]
        if len(maxima) == 1:
            return k[maxima[0]]
        
    def initialize_nb_dict(self):
        self.nb_dict = {}
        for label in self.labels:
            self.nb_dict[label] = defaultdict(list)
        
class NaiveBayes(NaiveBaseClass):
    def train(self, X, Y):
        self.labels = np.unique(Y)
        no_rows, no_cols = np.shape(X)
        self.initialize_nb_dict()
        self.class_probabilities = self.calculate_relative_occurences(Y)
        
        for label in self.labels:
            row_indices = np.where(Y==label)[0]
            X_ = X[row_indices,:]
            no_rows_, no_cols_ = np.shape(X_)
            for jj in range(0, no_cols_):
                self.nb_dict[label][jj] += list(X_[:,jj])
#         print(self.nb_dict)
#         print('\n')
        for label in self.labels:
            for jj in range(0, no_cols):
                self.nb_dict[label][jj] = self.calculate_relative_occurences(self.nb_dict[label][jj])
                
    def classify_single_elem(self, X_elem):
        Y_dict = {}
        for label in self.labels:
            class_probability = self.class_probabilities[label]
            for ii in range(0,len(X_elem)):
                relative_feature_values = self.nb_dict[label][ii]
                if X_elem[ii] in relative_feature_values.keys():
                    class_probability *= relative_feature_values[X_elem[ii]]
                else:
                    class_probability *= 0
                
            Y_dict[label] = class_probability
        return self.get_max_value_key(Y_dict)
        
    def classify(self,X):
        self.predicted_y_values = []
        no_rows, no_cols = np.shape(X)
        for ii in range(0,no_rows):
            X_elem = X[ii,:]
            prediction = self.classify_single_elem(X_elem)
            self.predicted_y_values.append(prediction)
        return self.predicted_y_values
    

### Train and predict

In [19]:
X_train, Y_train = get_pima_data()
nbc = NaiveBayes()
nbc.train(X_train, Y_train)
preds = nbc.classify(X_train)
print('Accuracy: {0}'.format((preds == Y_train).sum().astype(float) / len(preds)))

Accuracy: 0.9791666666666666
